# Distinguishing non-regular graphs

There are instances of graphs that are not *k*-regular nor isomorphic and yet
are not distinguishable via the message passing GNNs when their nodes
have identical features \cite{gnnpower}. An example of such graphs
is shown in Figure \ref{fig:nonregular}.
In PyNeuraLogic, we are capable of distinguishing those graphs,
for example, via the previously proposed model (\ref{lst:disttria})
which captures triangles of graph _a_ to distinguish between graphs.

In [1]:
from neuralogic.nn import get_evaluator
from neuralogic.core import Backend
from neuralogic.core import Atom, Template, Var, Term
from neuralogic.core.settings import Settings, Optimizer
from neuralogic.utils.data import Dataset

In [2]:
settings = Settings(optimizer=Optimizer.SGD, epochs=200)
train_dataset = Dataset()

with Template(settings).context() as template:
    template.add_rules([
        # Captures triangle
        Atom.triangle(Var.X)[1,] <= (
            Atom.edge(Var.X, Var.Y), Atom.feature(Var.Y)[1,],
            Atom.edge(Var.Y, Var.Z), Atom.feature(Var.Z)[1,],
            Atom.edge(Var.Z, Var.X), Atom.feature(Var.X)[1,],
        ),

        # Captures general graph
        Atom.general(Var.X)[1,] <= (Atom.edge(Var.X, Var.Y), Atom.feature(Var.Y)[1,]),
        Atom.general(Var.X)[1,] <= Atom.feature(Var.Y)[1,],

        Atom.predict <= Atom.general(Var.X)[1,],
        Atom.predict <= Atom.triangle(Var.X)[1,],
    ])

    # Encoding of graph a)
    train_dataset.add_example(
        [
            Atom.edge(1, 2), Atom.edge(2, 3), Atom.edge(3, 1), Atom.edge(2, 4),
            Atom.edge(4, 5), Atom.edge(5, 6), Atom.edge(6, 4),
            Atom.edge(2, 1), Atom.edge(3, 2), Atom.edge(1, 3), Atom.edge(4, 2),
            Atom.edge(5, 4), Atom.edge(6, 5), Atom.edge(4, 6),

            Atom.feature(1), Atom.feature(2), Atom.feature(3),
            Atom.feature(4), Atom.feature(5), Atom.feature(6),
        ],
    )

    # Encoding of graph b)
    train_dataset.add_example(
        [
            Atom.edge(1, 2), Atom.edge(2, 3), Atom.edge(3, 4), Atom.edge(4, 1),
            Atom.edge(2, 5), Atom.edge(5, 6), Atom.edge(6, 3),
            Atom.edge(2, 1), Atom.edge(3, 2), Atom.edge(4, 3), Atom.edge(1, 4),
            Atom.edge(5, 2), Atom.edge(6, 5), Atom.edge(3, 6),

            Atom.feature(1), Atom.feature(2), Atom.feature(3),
            Atom.feature(4), Atom.feature(5), Atom.feature(6),
        ],
    )

    train_dataset.add_queries([
        Atom.predict[1],
        Atom.predict[0],
    ])

In [3]:
neuralogic_evaluator = get_evaluator(Backend.DYNET, template)

for _ in neuralogic_evaluator.train(train_dataset):
    pass

graphs = ["a", "b"]

for graph_id, (label, predicted) in enumerate(neuralogic_evaluator.test(train_dataset)):
    print(f"Graph {graphs[graph_id]} is predicted to be class: {int(round(predicted))} | {predicted}")

Graph a is predicted to be class: 1 | 0.9537661075592041
Graph b is predicted to be class: 0 | 0.05284144729375839


Another interesting approach of a slightly different extension
of vanilla GNNs might be capturing based on the structure and the
cardinality of nodes. We can add additional information about the
cardinality of each node into examples, for instance, as atoms with
predicate's name *cardinality* with two terms -
the node id and its cardinality. We can then choose which atom will
be aggregated based on its cardinality to distinguish graph _a_ and graph *b*, as shown in Example 2, where we capture only sub-graphs of graphs

The `a_graph` captures a triangle (`Var.X`, `Var.Y`, `Var.Z`)
connected to one node (`Var.T`) with a cardinality of three.
In contrast, the `b_graph` captures a cycle of length of four
 (`Var.X`, `Var.Y`, `Var.Z`, `Var.T`)
 which has to satisfy required cardinalities.


#### Example 2: Distinguishing between graphs based on their cardinality

In [17]:
settings = Settings(optimizer=Optimizer.SGD, epochs=200)
train_dataset = Dataset()

with Template(settings).context() as template:
    template.add_rules([
        Atom.a_graph(Var.X) <= (
            Atom.edge(Var.X, Var.Y), Atom.cardinality(Var.Y, 2)[1,],
            Atom.edge(Var.Y, Var.Z), Atom.cardinality(Var.Z, 2)[1,],
            Atom.edge(Var.Z, Var.X), Atom.cardinality(Var.X, 3)[1,],
            Atom.edge(Var.X, Var.T), Atom.cardinality(Var.T, 3)[1,],
            Atom.special.alldiff(...),
        ),
        Atom.b_graph(Var.X) <= (
            Atom.edge(Var.X, Var.Y), Atom.cardinality(Var.Y, 2)[1,],
            Atom.edge(Var.Y, Var.Z), Atom.cardinality(Var.Z, 2)[1,],
            Atom.edge(Var.Z, Var.T), Atom.cardinality(Var.T, 3)[1,],
            Atom.edge(Var.T, Var.X), Atom.cardinality(Var.X, 3)[1,],
            Atom.special.alldiff(...),
        ),
        Atom.predict <= Atom.a_graph(Var.X)[1,],
        Atom.predict <= Atom.b_graph(Var.X)[1,],
    ])

    # Encoding of graph a)
    train_dataset.add_example(
        [
            Atom.edge(1, 2), Atom.edge(2, 3), Atom.edge(3, 1), Atom.edge(2, 4),
            Atom.edge(4, 5), Atom.edge(5, 6), Atom.edge(6, 4),
            Atom.edge(2, 1), Atom.edge(3, 2), Atom.edge(1, 3), Atom.edge(4, 2),
            Atom.edge(5, 4), Atom.edge(6, 5), Atom.edge(4, 6),

            Atom.cardinality(1, 2), Atom.cardinality(2, 3), Atom.cardinality(3, 2),
            Atom.cardinality(4, 3), Atom.cardinality(5, 2), Atom.cardinality(6, 2),
        ],
    )

    # Encoding of graph b)
    train_dataset.add_example(
        [
            Atom.edge(1, 2), Atom.edge(2, 3), Atom.edge(3, 4), Atom.edge(4, 1),
            Atom.edge(2, 5), Atom.edge(5, 6), Atom.edge(6, 3),
            Atom.edge(2, 1), Atom.edge(3, 2), Atom.edge(4, 3), Atom.edge(1, 4),
            Atom.edge(5, 2), Atom.edge(6, 5), Atom.edge(3, 6),

            Atom.cardinality(1, 2), Atom.cardinality(2, 3), Atom.cardinality(3, 3),
            Atom.cardinality(4, 2), Atom.cardinality(5, 2), Atom.cardinality(6, 2),
        ],
    )

    train_dataset.add_queries([
        Atom.predict[1],
        Atom.predict[0],
    ])

In [18]:
neuralogic_evaluator = get_evaluator(Backend.DYNET, template)

for _ in neuralogic_evaluator.train(train_dataset):
    pass

graphs = ["a", "b"]

for graph_id, (label, predicted) in enumerate(neuralogic_evaluator.test(train_dataset)):
    print(f"Graph {graphs[graph_id]} is predicted to be class: {int(round(predicted))} | {predicted}")

Graph a is predicted to be class: 1 | 0.965618371963501
Graph b is predicted to be class: 0 | -2.301345969587176e-19


Figure X. shows two graphs, _a_
and _b_,
representing a real-world structure of two molecules _Bicyclopentyl_
and *Decalin*, respectively. The message passing GNN cannot again distinguish between
graphs under the condition of identical features for all nodes \cite{gnnpower}.
In PyNeuraLogic, we can embed, for example, the cycle of length five present in
graph _a_ and thus distinguish those instances, such as is shown in
Example \ref{lst:distfive}.


#### Example 3: Capturing the cycle of the length of five

In [25]:
settings = Settings(optimizer=Optimizer.SGD, epochs=200)
train_dataset = Dataset()

with Template(settings).context() as template:
    template.add_rules([
        # Captures cycle of the length of five (Bicyclopentyl)
        Atom.cycle_of_the_length_of_five(Var.X)[1,] <= (
            Atom.edge(Var.X, Var.Y), Atom.feature(Var.Y)[1,],
            Atom.edge(Var.Y, Var.Z), Atom.feature(Var.Z)[1,],
            Atom.edge(Var.Z, Var.R), Atom.feature(Var.R)[1,],
            Atom.edge(Var.R, Var.S), Atom.feature(Var.S)[1,],
            Atom.edge(Var.S, Var.X), Atom.feature(Var.X)[1,],
            Atom.special.alldiff(...),
        ),

        # Captures general graph (such as Decalin)
        Atom.general(Var.X)[1,] <= (Atom.edge(Var.X, Var.Y), Atom.feature(Var.Y)[1,]),
        Atom.general(Var.X)[1,] <= Atom.feature(Var.Y)[1,],

        Atom.predict <= Atom.general(Var.X)[1,],
        Atom.predict <= Atom.cycle_of_the_length_of_five(Var.X)[1,],
    ])

    # Encoding of graph Bicyclopentyl
    train_dataset.add_example(
        [
            Atom.edge(1, 2), Atom.edge(2, 3), Atom.edge(3, 4), Atom.edge(4, 5), Atom.edge(5, 1), Atom.edge(1, 6),
            Atom.edge(2, 1), Atom.edge(3, 2), Atom.edge(4, 3), Atom.edge(5, 4), Atom.edge(1, 5), Atom.edge(6, 1),
            Atom.edge(6, 7), Atom.edge(7, 8), Atom.edge(8, 9), Atom.edge(9, 10), Atom.edge(10, 6),
            Atom.edge(7, 6), Atom.edge(8, 7), Atom.edge(9, 8), Atom.edge(10, 9), Atom.edge(6, 10),

            Atom.feature(1), Atom.feature(2), Atom.feature(3), Atom.feature(4), Atom.feature(5),
            Atom.feature(6), Atom.feature(7), Atom.feature(8), Atom.feature(9), Atom.feature(10),
        ],
    )

    # Encoding of graph Decalin
    train_dataset.add_example(
        [
            Atom.edge(1, 2), Atom.edge(2, 3), Atom.edge(3, 4), Atom.edge(4, 5), Atom.edge(5, 6), Atom.edge(1, 6),
            Atom.edge(2, 1), Atom.edge(3, 2), Atom.edge(4, 3), Atom.edge(5, 4), Atom.edge(6, 5), Atom.edge(6, 1),
            Atom.edge(6, 7), Atom.edge(7, 8), Atom.edge(8, 9), Atom.edge(9, 10), Atom.edge(10, 1),
            Atom.edge(7, 6), Atom.edge(8, 7), Atom.edge(9, 8), Atom.edge(10, 9), Atom.edge(1, 10),

            Atom.feature(1), Atom.feature(2), Atom.feature(3), Atom.feature(4), Atom.feature(5),
            Atom.feature(6), Atom.feature(7), Atom.feature(8), Atom.feature(9), Atom.feature(10),
        ],
    )

    train_dataset.add_queries([
        Atom.predict[1],
        Atom.predict[0],
    ])

In [27]:
neuralogic_evaluator = get_evaluator(Backend.DYNET, template)

for _ in neuralogic_evaluator.train(train_dataset):
    pass

graphs = ["Bicyclopentyl", "Decalin"]

for graph_id, (label, predicted) in enumerate(neuralogic_evaluator.test(train_dataset)):
    print(f"Graph {graphs[graph_id]} is predicted to be class: {int(round(predicted))} | {predicted}")


Graph Bicyclopentyl is predicted to be class: 1 | 0.9578434228897095
Graph Decalin is predicted to be class: 0 | 0.04274271801114082
